In [1]:
import pandas as pd

In [2]:
from sklearn.ensemble import RandomForestRegressor 
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import  mean_squared_error
from keras.models import Sequential
from keras.layers import Dense
import numpy as np

C:\Users\Arpita\Anaconda3\lib\site-packages\sklearn\ensemble\weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d
C:\Users\Arpita\Anaconda3\lib\site-packages\h5py\__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [3]:
cdsSpread_data = pd.read_stata("C:\\Users\\Arpita\\Desktop\\Book\\Fall 2018\\Machine Learning\\Assignment\\cds_spread5y_2001_2016.dta")


In [5]:
cdsSpread_data.shape

(169043, 9)

In [6]:
cdsSpread_data.dtypes

gvkey                    object
mdate            datetime64[ns]
redcode                  object
tier                     object
ticker                   object
shortname                object
docclause                object
spread5y                float64
impliedrating            object
dtype: object

In [7]:
cdsSpread_data.head()

,gvkey,mdate,redcode,tier,ticker,shortname,docclause,spread5y,impliedrating
0,001038,2004-08-31,0A17CF,SNRFOR,AEN,AMC Entmt Inc,MR,0.042000,BB
1,001038,2005-08-31,0A17CF,SNRFOR,AEN,AMC Entmt Inc,XR,0.034000,BB
2,001038,2005-09-30,0A17CF,SNRFOR,AEN,AMC Entmt Inc,XR,0.034000,BB
3,001038,2005-10-31,0A17CF,SNRFOR,AEN,AMC Entmt Inc,XR,0.034001,BB
4,001038,2005-11-30,0A17CF,SNRFOR,AEN,AMC Entmt Inc,XR,0.034001,BB


In [8]:
#Seperate date based on month & years and create quarters accordingly 
cdsSpread_data['Date'] = pd.to_datetime(cdsSpread_data['mdate'])
cdsSpread_data['Month']= cdsSpread_data['Date'].dt.month
cdsSpread_data['Year']=cdsSpread_data['Date'].dt.year

cdsSpread_data['quarter']='4'


In [9]:
cdsSpread_data.loc[cdsSpread_data['Month']>9,"quarter"]=4
cdsSpread_data.loc[(cdsSpread_data['Month']>6) & (cdsSpread_data['Month']<=9),"quarter"]=3
cdsSpread_data.loc[(cdsSpread_data['Month']>3) & (cdsSpread_data['Month']<=6),"quarter"]=2
cdsSpread_data.loc[(cdsSpread_data['Month'])<=3,"quarter"]=1

In [10]:
cdsSpread_data['gvkey'] = cdsSpread_data['gvkey'].astype(float)
cdsSpread_data['quarter'] = cdsSpread_data['quarter'].astype(float)
cdsSpread_data['Year'] = cdsSpread_data['Year'].astype(float)

In [11]:
qtr_Mdata = pd.read_csv("C:\\Users\\Arpita\\Downloads\\Quarterly Merged CRSP-Compustat.csv", low_memory = False)

In [12]:
qtr_Mdata.shape

(58928, 687)

In [13]:
qtr_Mdata.dtypes

GVKEY          int64
LINKPRIM      object
LIID          object
LINKTYPE      object
LPERMNO        int64
LPERMCO        int64
LINKDT        object
LINKENDDT     object
datadate      object
fyearq         int64
fqtr           int64
fyr            int64
indfmt        object
consol        object
popsrc        object
datafmt       object
tic           object
cusip         object
conm          object
acctchgq      object
acctstdq      object
adrrq        float64
ajexq        float64
ajpq         float64
bsprq         object
compstq       object
curcdq        object
curncdq       object
currtrq      float64
curuscnq     float64
              ...   
addzip        object
busdesc       object
city          object
conml         object
county        object
dlrsn        float64
ein           object
fax           object
fyrc           int64
ggroup         int64
gind           int64
gsector        int64
gsubind        int64
idbflag       object
incorp        object
loc           object
naics        

In [14]:
qtr_Mdata.head()

,GVKEY,LINKPRIM,LIID,LINKTYPE,LPERMNO,LPERMCO,LINKDT,LINKENDDT,datadate,fyearq,...,priusa,sic,spcindcd,spcseccd,spcsrc,state,stko,weburl,dldte,ipodate
0,1038,P,01,LU,66413,6301,08/18/1983,12/31/2004,03/31/2001,2000,...,1,7830,245.0,978.0,NaN,MO,0.0,www.amctheatres.com,12/27/2004,NaN
1,1038,P,01,LU,66413,6301,08/18/1983,12/31/2004,06/30/2001,2001,...,1,7830,245.0,978.0,NaN,MO,0.0,www.amctheatres.com,12/27/2004,NaN
2,1038,P,01,LU,66413,6301,08/18/1983,12/31/2004,09/30/2001,2001,...,1,7830,245.0,978.0,NaN,MO,0.0,www.amctheatres.com,12/27/2004,NaN
3,1038,P,01,LU,66413,6301,08/18/1983,12/31/2004,12/31/2001,2001,...,1,7830,245.0,978.0,NaN,MO,0.0,www.amctheatres.com,12/27/2004,NaN
4,1038,P,01,LU,66413,6301,08/18/1983,12/31/2004,03/31/2002,2001,...,1,7830,245.0,978.0,NaN,MO,0.0,www.amctheatres.com,12/27/2004,NaN


In [15]:
qtr_Mdata=qtr_Mdata.rename(columns = {'GVKEY':'gvkey'})
qtr_Mdata=qtr_Mdata.rename(columns = {'datadate':'mdate'})

In [16]:
qtr_Mdata['mdate'].unique()

array(['03/31/2001', '06/30/2001', '09/30/2001', '12/31/2001',
       '03/31/2002', '06/30/2002', '09/30/2002', '12/31/2002',
       '03/31/2003', '06/30/2003', '09/30/2003', '12/31/2003',
       '03/31/2004', '06/30/2004', '09/30/2004', '12/31/2004',
       '03/31/2005', '06/30/2005', '09/30/2005', '12/31/2005',
       '03/31/2006', '06/30/2006', '09/30/2006', '12/31/2006',
       '03/31/2007', '06/30/2007', '09/30/2007', '12/31/2007',
       '03/31/2008', '06/30/2008', '09/30/2008', '12/31/2008',
       '03/31/2009', '06/30/2009', '09/30/2009', '12/31/2009',
       '03/31/2010', '06/30/2010', '09/30/2010', '12/31/2010',
       '03/31/2011', '06/30/2011', '09/30/2011', '12/31/2011',
       '12/31/2013', '03/31/2014', '06/30/2014', '09/30/2014',
       '12/31/2014', '03/31/2015', '06/30/2015', '09/30/2015',
       '12/31/2015', '03/31/2016', '06/30/2016', '09/30/2016',
       '12/31/2016', '03/31/2017', '03/31/2012', '06/30/2012',
       '09/30/2012', '12/31/2012', '03/31/2013', '06/30

In [17]:
#Seperate date based on month & years and create quarters accordingly 
qtr_Mdata['Date'] = pd.to_datetime(qtr_Mdata['mdate'])
qtr_Mdata['Month']= qtr_Mdata['Date'].dt.month
qtr_Mdata['Year']=qtr_Mdata['Date'].dt.year

qtr_Mdata['quarter']='4'


In [18]:
qtr_Mdata.loc[qtr_Mdata['Month']>9,"quarter"]=4
qtr_Mdata.loc[(qtr_Mdata['Month']>6) & (qtr_Mdata['Month']<=9),"quarter"]=3
qtr_Mdata.loc[(qtr_Mdata['Month']>3) & (qtr_Mdata['Month']<=6),"quarter"]=2
qtr_Mdata.loc[(qtr_Mdata['Month'])<=3,"quarter"]=1

In [19]:
qtr_Mdata['gvkey'] = qtr_Mdata['gvkey'].astype(float)
qtr_Mdata['quarter'] = qtr_Mdata['quarter'].astype(float)
qtr_Mdata['Year'] = qtr_Mdata['Year'].astype(float)

In [20]:
qtr_Mdata['quarter'].head()

0    1.0
1    2.0
2    3.0
3    4.0
4    1.0
Name: quarter, dtype: float64

In [36]:
data = pd.merge(cdsSpread_data, qtr_Mdata, on=['gvkey', 'quarter', 'Year'])

In [37]:
#Use only numerical records 
numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
data = data.select_dtypes(include=numerics)

#Pre processing data: replace NA with median 
data=data.fillna(data.median())
data=data.dropna(axis=1, how='any')

X=data.drop('spread5y', axis=1)

y=data['spread5y']
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

X_train

,gvkey,Month_x,Year,quarter,LPERMNO,LPERMCO,fyearq,fqtr,fyr,adrrq,...,gind,gsector,gsubind,naics,priusa,sic,spcindcd,spcseccd,stko,Month_y
50630,9466.0,4,2007.0,2.0,77354,11358,2006,4,5,1.0,...,254010,25,25401040,511130,1,2731,410.0,976.0,0.0,5
35365,6799.0,11,2004.0,4.0,24360,21116,2004,4,12,1.0,...,551030,55,55103010,22111,1,4931,705.0,700.0,0.0,12
3413,1602.0,3,2011.0,1.0,14008,216,2011,1,12,1.0,...,352010,35,35201010,325414,1,2836,147.0,905.0,0.0,3
71256,14624.0,6,2006.0,2.0,75573,9798,2006,2,12,1.0,...,255040,25,25504040,453210,1,5940,449.0,976.0,0.0,6
11223,2593.0,1,2017.0,1.0,46690,20374,2017,2,9,1.0,...,151010,15,15101010,325180,1,2890,160.0,970.0,0.0,3
95769,64166.0,7,2015.0,3.0,84373,31838,2015,3,12,1.0,...,351020,35,35102015,621511,1,8071,292.0,905.0,0.0,9
62229,11781.0,1,2005.0,1.0,89134,42001,2005,1,12,1.0,...,101020,10,10102020,211111,1,1311,355.0,925.0,0.0,3
39461,7750.0,7,2015.0,3.0,52337,21258,2015,3,12,1.0,...,351020,35,35102020,622110,1,8062,295.0,905.0,0.0,9
26869,5234.0,12,2009.0,4.0,16432,20834,2009,4,12,1.0,...,251010,25,25101020,326211,1,3011,130.0,976.0,0.0,12
97645,100039.0,2,2004.0,1.0,68823,3977,2003,4,3,5.0,...,252010,25,25201010,333313,90,3579,370.0,925.0,3.0,3


In [38]:
X_test= X_test.drop('Month_x', axis=1)
X_test= X_test.drop('Month_y', axis=1)
X_test= X_test.drop('quarter', axis=1)
X_test= X_test.drop('Year', axis=1)
X_test= X_test.drop('gvkey', axis=1)

TrainData=data[(data['Year'] < 2010)]

#splitting x and y for test data
X_train= TrainData.drop('spread5y', axis=1)
y_train=TrainData['spread5y']
X_train= X_train.drop('Month_x', axis=1)
X_train= X_train.drop('Month_y', axis=1)
X_train= X_train.drop('quarter', axis=1)
X_train= X_train.drop('Year', axis=1)
X_train= X_train.drop('gvkey', axis=1)

In [34]:
TestData.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 32 entries, 95 to 126
Columns: 473 entries, gvkey to Month_y
dtypes: float64(455), int64(18)
memory usage: 118.5 KB


In [39]:
randforst = RandomForestRegressor(n_estimators=50) 
randforst.fit(X_train, y_train)
#randforst.score(X_test, y_test)
randforst_Pred=randforst.predict(X_test)

In [40]:
F_imp=randforst.feature_importances_
F_imp = pd.DataFrame(randforst.feature_importances_,index = X_train.columns,columns=['imp']).sort_values('imp',ascending=False)
newfeatures=F_imp.iloc[:50,:]
newfeatures=newfeatures.index.tolist()

def mean_absolute_percentage_error(y_true, y_pred): 
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

mean_squared_error(y_test,randforst_Pred)
mean_absolute_percentage_error(y_test, randforst_Pred)

#Dataset with only new features
X_train_NewF=X_train[newfeatures]
X_test_NewF=X_test[newfeatures]


scaler = StandardScaler()
scaler.fit(X_train_NewF)
StandardScaler(copy=True, with_mean=True, with_std=True)
X_train_NewFt=scaler.transform(X_train_NewF)
X_test_NewFt=scaler.transform(X_test_NewF)

In [43]:
np.random.seed(10)
classifier = Sequential()
classifier.add(Dense(32, input_dim=50, activation='relu'))
classifier.add(Dense(8, activation='elu'))
classifier.add(Dense(1, activation='relu')) 
classifier.compile(optimizer='adam',loss='mse',  metrics=['accuracy'])
classifier.fit(X_train_NewFt, y_train, batch_size=10 epochs=100,)

Epoch 1/100
66774/66774 [==============================] - 8s 116us/step - loss: 0.0109 - acc: 0.0000e+00
Epoch 2/100
66774/66774 [==============================] - 7s 107us/step - loss: 0.0017 - acc: 0.0000e+00
Epoch 3/100
66774/66774 [==============================] - 6s 91us/step - loss: 0.0014 - acc: 0.0000e+00
Epoch 4/100
66774/66774 [==============================] - 7s 106us/step - loss: 0.0013 - acc: 0.0000e+00
Epoch 5/100
66774/66774 [==============================] - 6s 95us/step - loss: 0.0016 - acc: 0.0000e+00
Epoch 6/100
66774/66774 [==============================] - 6s 88us/step - loss: 0.0011 - acc: 0.0000e+00
Epoch 7/100
66774/66774 [==============================] - 6s 95us/step - loss: 0.0010 - acc: 0.0000e+00
Epoch 8/100
66774/66774 [==============================] - 6s 90us/step - loss: 0.0011 - acc: 0.0000e+00
Epoch 9/100
66774/66774 [==============================] - 6s 92us/step - loss: 9.7355e-04 - acc: 0.0000e+00
Epoch 10/100
66774/66774 [======================

66774/66774 [==============================] - 7s 98us/step - loss: 6.6387e-04 - acc: 0.0000e+00
Epoch 76/100
66774/66774 [==============================] - 6s 93us/step - loss: 6.8023e-04 - acc: 0.0000e+00
Epoch 77/100
66774/66774 [==============================] - 6s 95us/step - loss: 6.4276e-04 - acc: 0.0000e+00
Epoch 78/100
66774/66774 [==============================] - 6s 85us/step - loss: 6.5693e-04 - acc: 0.0000e+00
Epoch 79/100
66774/66774 [==============================] - 6s 86us/step - loss: 6.6422e-04 - acc: 0.0000e+00
Epoch 80/100
66774/66774 [==============================] - 6s 96us/step - loss: 6.9277e-04 - acc: 0.0000e+00
Epoch 81/100
66774/66774 [==============================] - 6s 87us/step - loss: 6.5351e-04 - acc: 0.0000e+00
Epoch 82/100
66774/66774 [==============================] - 6s 89us/step - loss: 6.6153e-04 - acc: 0.0000e+00
Epoch 83/100
66774/66774 [==============================] - 6s 93us/step - loss: 6.5948e-04 - acc: 0.0000e+00
Epoch 84/100
66774/6677

In [ ]:
#Calculate mean absolute error
Y_pred = model.predict(X_test_NewFt)

mape=mean_absolute_percentage_error(y_test,Y_pred)

mape